# Quantum Software Development Journey: 
# From Theory to Application with Classiq - Part 1

**Welcome to the Classiq Workshop Series for QClass 2024!**

In this series, we will develop the skills needed to participate in quantum software development!

- Week 1: Classiq's Basics & High-Level Functional Design
- Week 2: Using Git as a Tool for In-Team Collaboration and Open Source Contributions
- Weeks 3-4: Advanced Algorithms, Introduction to Quantum Machine Learning (QML), and Their Applications

**Here, you have early access to our [New Classiq's documentation](https://nightly.docs.classiq.io/latest/)!**




Additional resources you should use are
- The IDE of the classiq platform at [platform.classiq.io](platform.classiq.io)
- The [community Slack of Classiq](https://short.classiq.io/join-slack) - Classiq's team will answer any question you have over there, including implementation questions
- [Classiq's documentation](https://docs.classiq.io/latest/user-guide/platform/) with the dedicated [Python SDK explanations](https://docs.classiq.io/latest/user-guide/platform/qmod/python/functions/)

Good luck!

## Setting The Scene

Install the Classiq SDK package:

In [10]:
# !pip install -U classiq

You need to authenticate your device in order to use Classiq's backend synthesis engine and IDE. 
**Make sure to register to the platform** at [platform.classiq.io](https://platform.classiq.io/) before you run the next cell:

In [11]:
import classiq
# classiq.authenticate()

In [2]:
from classiq import *

## A Warm Up

### First Example

Write a function that prepares the minus state $\ket{-}=\frac{1}{\sqrt2}(\ket{0}-\ket{1})$, assuming it recives the qubit $\ket{x}=\ket{0}$ (hint): 

<details>
<summary>
HINT
</summary>

Use `H(x)`,`X(x)`
</details>

In [13]:
@qfunc
def prepare_minus_state(x:QBit):
    pass #TODO delete pass
    #TODO prepare |-> function

Now we will test our code:

In [14]:
@qfunc
def main(x: Output[QBit]):
    allocate(1,x) # Initialize the qubit x
    pass #TODO delete pass
    #TODO apply the function prepare_minus

In [15]:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

In [16]:
show(quantum_program)

Opening: https://platform.classiq.io/circuit/185d88cf-237a-4991-acb9-802f796c99ee?version=0.40.0


### Uniform Superposition

Let's continue warming up with creating a function that receives a quantum register and creates a uniform superposition for all qubits within this array. You should use the function `apply_to_all(gate_operand=, target=)`:

In [17]:
@qfunc
def create_initial_state(reg: QArray[QBit]):
    pass #TODO delete pass
    #TODO use the function apply_to_all in order create a uniform superposition

Test your function by creating a new main function, synthesizing and viewing the circuit:

In [39]:
@qfunc
def main(): #TODO fill in the correct declaration here, what variables this model should output?
    pass #TODO delete pass
    #TODO allocate reg with a few qubits
    create_initial_state(reg)

In [40]:
# TODO uncomment the following line:
# qprog = synthesize(create_model(main))

#TODO show the quantum program

Another implementation could utilize the `repeat(count=, iteration=)` function. The `repeat` function can be thought of as a "classical for loop". It could be handy in many situations, especially when combined with `if_` function, the "classical if" statement in Classiq. Together, they form 'Classical Control Flow'. 

Read more: [Classical Control Flow (repeat, if_)](https://docs.classiq.io/latest/user-guide/platform/qmod/language-reference/statements/classical-control-flow/?h=repea#__tabbed_1_2)

In [41]:
@qfunc
def create_initial_state(q: QArray[QBit]) -> None:
    repeat(q.len, lambda i: H(q[i]))

@qfunc
def main(reg: Output[QArray]): 
    allocate(4,reg)
    create_initial_state(reg)

qprog = synthesize(create_model(main))
show(qprog)

Opening: https://platform.classiq.io/circuit/9ef28f93-59fe-467a-8b6d-efd7a0648fce?version=0.40.0


## Guidelines for High-Level Functional Design with Classiq

**Some basic explanations about the high-level functional design with Classiq:**

* There should always be a main (`def main(...)`) function - the model that captures your algorithm is described there

* The model is always generated out of the main function 

* The model is sent to the synthesis engine (compiler) that return a quantum program which contains the quantum circuit

**Some basic guidelines about the modeling language (QMOD):**

1. Every function you use with the QMOD language should have the decorator `@qfunc` before it
2. Every quantum variable should be declared, either as an argument of a function e.g. `def prepare_minus(x: QBit)` or as a local variable within the function itself with `x = QBit('x')`


3. Some quantum variables need to be initialized with the `allocate` function. This is required in 2 cases:
* A variable is an argument of a function with the declaration `Output` like `def main(x: Output[QNum])`
* A variable that was declared within a function like `a = QNum('a')`

4. For the `main` function, you will always use `Output` for all variables. The `output` indicates that these quantum variables are not initialized outside the scope of the function.


<details> 
<summary> Types of Initializations </summary>
There are a few ways to initialize a quantum variable:

1. With `allocate` or `allocate_num` 
2. With `prepare_int`, `prepare_state` or `prepare_amplitudes`
3. As the result of a numeric operation `|=`
4. With the `bind` operation (`->` in native)
5. With any function that declares its quantum variable argument as `output`

</details>

<details> 
<summary> Types of Quantum Variables </summary>
In Qmod there are 3 types of quantum variables:

1. `QBit` (`qbit`)
2. `QArray[QBit]` (`qbit[]`)
3. `QNum` (`qnum`)

(See also [Quantum Variables](https://nightly.docs.classiq.io/latest/classiq_101/classiq_concepts/design/quantum_variables_and_functions/))
</details>

## Tutorial - State Preparation 

### Prepare State

Now, we will see how we can easily make arbitrary state using Classiq's `prepare_state`.

For example, let’s say we want to prepare the state $ \ket{\Phi^+}=\frac{1}{\sqrt{2}}(\ket{00}+\ket{11}) $

In [26]:
@qfunc
def main(x: Output[QArray[QBit]]):
    prepare_state(probabilities=[0.5,0,0,0.5], bound=0.01, out=x)

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Opening: https://platform.classiq.io/circuit/9e0cede6-2313-4bf2-a1c8-884b0ce731d7?version=0.40.0


Or using `prepare_bell_state` in order to prepare states with relative phase:

In [27]:
@qfunc
def main(x:Output[QArray]):
    prepare_bell_state(state_num=0, q=x) # phi-

In [28]:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/b172cbea-d4a9-468e-9323-6159e9f0fb88?version=0.40.0


<details>
<summary>
NOTE
</summary>


| State Number | Bell State | 
|--------------|------------|
| 0            | $ \ket{\phi^+}= \frac{1}{\sqrt{2}}[\ket{00}+\ket{11}] $  | 
| 1            | $ \ket{\phi^-}= \frac{1}{\sqrt{2}}[\ket{00}-\ket{11}]$  |
| 2            | $ \ket{\psi^+}= \frac{1}{\sqrt{2}}[\ket{01}+\ket{10}]$  |
| 3            | $ \ket{\psi^-}= \frac{1}{\sqrt{2}}[\ket{01}-\ket{10}]$  |
| 4=0          | $ \ket{\phi^+}= ...$  |
| ...          | ...        | ...   | 


</details>

Now it's your turn to prepare the state $ \ket{\Psi^+}=\frac{1}{\sqrt{2}}(\ket{01}+\ket{10}) $

In [30]:
@qfunc
def main(x: Output[QArray[QBit]]):
    pass #TODO delete pass
    #TODO use the function prepare_state/ prepare_bell_state in order create a the desired state

#TODO create model, synthesize it, and show it

Note that we can use `prepare_state` to create much more complex states, for instance, we can create list of probabilities:

In [31]:
probabilities = [
    0,
    0.002,
    0.004,
    0.006,
    0.0081,
    0.0101,
    0.0121,
    0.0141,
    0.0161,
    0.0181,
    0.0202,
    0.0222,
    0.0242,
    0.0262,
    0.0282,
    0.0302,
    0.0323,
    0.0343,
    0.0363,
    0.0383,
    0.0403,
    0.0423,
    0.0444,
    0.0464,
    0.0484,
    0.0504,
    0.0524,
    0.0544,
    0.0565,
    0.0585,
    0.0605,
    0.0625,
]

And then generate the state:

In [32]:
@qfunc
def main(x: Output[QArray[QBit]]):
    prepare_state(probabilities=probabilities, bound=0.01, out=x)
    
model = create_model(main)
qprog = synthesize(model)
show(qprog)

Opening: https://platform.classiq.io/circuit/42118092-b20e-4172-98c6-00943a067da4?version=0.40.0


### Prepare Int

We also have a `prepare_int` function, which allows us to register integers effortlessly. For example, $binary(9) = 1001$, or any other integer can be prepared in a single line of code:

In [33]:
from classiq import *

@qfunc
def main(x: Output[QNum]) -> None:
    prepare_int(9, x)

model = create_model(main)
qprog = synthesize(model)
show(qprog)

Opening: https://platform.classiq.io/circuit/57617572-0d31-42e6-88c4-a54efb0d7b47?version=0.40.0


### Exercise: Parallel Addition 

In this exercise, we will conclude state preparation and also get a teaser of the next part of the tutorial - Arithmetic Operations. 

Let's say that for some reason we used `prepare_int` to create the integer 7, and we want to perform addition operations with the integers 0, 4, and 7. We will do that using `prepare_state`.

In [4]:
@qfunc
def main(res: Output[QNum]) -> None:
    pass #TODO delete pass
    x = QNum() #TODO complete the declarations of x,y
    y = QNum()
    
    # TODO prepare the above-mentioned integer using prepare_int
    prepare_state(probabilities=[], bound=0.01, out=y) # TODO complete the 'probabilities' list
    
    res|=x+y



In [6]:
# TODO uncomment the folllowing lines:

# model = create_model(main)
# qprog = synthesize(model)
# show(qprog)

## Tutorial - Arithmetic Operations with Classiq

One of the key advantages of Classiq is it's simplistic and powerful compiler for quantum arithmetic. Let's see an example:

In [ ]:
num_qubits = 4
fraction_digits = 0 
is_signed = True

@qfunc
def main(x: Output[QNum], y: Output[QNum]):
    allocate_num(num_qubits=num_qubits, is_signed=is_signed, fraction_digits=fraction_digits, out=x)
    hadamard_transform(x)
    y|= x**2 + 1

qmod = create_model(main)

The `allocate_num` function initializes a quantum variable that represent numbers. By default, it is initialized to the $\ket{0}$ state. Then the `hadmard_transform` creates a superposition of all possible states in the domain $[-2^3,2^3-1]$. Finally, the arithmetic operation creates the entangled superposition of states:
$\begin{equation}
\sum_{x =-2^3}^{2^3-1}\ket{x}\ket{x^2+1}.
\end{equation}$

The `qmod` variable is a string that captures the algorithm we have just created in a JSON format. Now, what we want is to synthesize (compile) in order to receive a concrete quantum program that contains the quantum circuit implementation.

In [ ]:
qprog = synthesize(qmod)
show(qprog)

### Advanced Arithmetics

Now let's create a general linear function with Classiq: $y= ax+b$ where $a,b$ are classical integer parameters and $x,y$ is a quantum states representing integers:

In [4]:
@qfunc
def linear_func(a:CInt,b: CInt, x:QNum, y: Output[QNum]):
    y |= a*x+b

In [5]:
@qfunc
def main(x:Output[QNum], y: Output[QNum]):

    a = 2
    b = 1
    allocate_num(num_qubits=4,is_signed=False,fraction_digits=0,out=x)
    hadamard_transform(x)
    linear_func(a,b,x,y)

qmod = create_model(main)

In [6]:
qprog = synthesize(qmod)

Let's execute the circuit from directly from the SDK:

In [7]:
job = execute(qprog)

And we can view the results in the IDE:

In [8]:
job.open_in_ide()

Or to directly analyze it within the SDK:

In [10]:
results = job.result()
parsed_counts = results[0].value.parsed_counts
for sampled_state in parsed_counts: print(sampled_state.state)

{'x': 8.0, 'y': 17.0}
{'x': 5.0, 'y': 11.0}
{'x': 13.0, 'y': 27.0}
{'x': 7.0, 'y': 15.0}
{'x': 14.0, 'y': 29.0}
{'x': 6.0, 'y': 13.0}
{'x': 2.0, 'y': 5.0}
{'x': 15.0, 'y': 31.0}
{'x': 1.0, 'y': 3.0}
{'x': 12.0, 'y': 25.0}
{'x': 10.0, 'y': 21.0}
{'x': 9.0, 'y': 19.0}
{'x': 11.0, 'y': 23.0}
{'x': 0.0, 'y': 1.0}
{'x': 4.0, 'y': 9.0}
{'x': 3.0, 'y': 7.0}


**Now it's your turn!** 

Implement the same linear function, but now $x$ is in the domain $[0,1)$ and is represented by 4 qubits. The parameters $a,b$ should be now `float` with the values of: $a=0.5, b=1.5$.

In [38]:
#TODO complete here

### Tutorial - Two controlled Linear operations

Let's say we want now to have two linear operations applied on the same quantum variable (register). But the arithmetic operation initialize a new quantum variable, so how can we do that? The answer is that we need to apply the operation to another variable and then XOR it to the variable we want. 

This can be useful if the linear operation we want to apply is controlled upon a control variable. Let's first define the functional building block:

In [11]:
@qfunc
def inplace_linear_attempt(a:CInt,b: CInt, x:QNum, y: QNum):
    tmp = QNum('tmp')
    linear_func(a,b,x,tmp)
    inplace_xor(tmp,y)

And checking our basic function implementation:

In [12]:
@qfunc
def main(x: Output[QNum],y: Output[QNum]):
    a = 1
    b = 2

    allocate_num(4,False,0,y)
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    inplace_linear_attempt(a,b,x,y)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/6e172347-e427-4622-84d0-01d46371a787?version=0.40.0


OK, cool. So now we want to add a control qubit that controlled on the state $\ket{0}$ implements the linear function $\ket{x}\rightarrow\ket{x}\ket{x+2}$ and controlled on the state $\ket{1}$ implements the linear function $\ket{x}\rightarrow\ket{x}\ket{2x+1}$:

In [13]:
@qfunc
def control_logic(a: CArray[CInt], b: CArray[CInt], controller:QNum, x: QNum, y: QNum):
    
    repeat( count=a.len,         
            iteration=lambda i: control(controller==i, lambda: inplace_linear_attempt(a[i],b[i],x,y)))


In [21]:
@qfunc
def main(controller: Output[QNum], x: Output[QNum],y: Output[QNum]):

    # Linear polynom parameters
    a = [1,2]
    b = [2,1]

    # Initializing x to a superposition in the domain [0,2^4-1]
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    
    #Initialize y
    allocate_num(4,False,0,y)

    # Setting the controller in a superposition
    allocate_num(1,False,0,controller)
    H(controller)

    # Implementing the control logic
    control_logic(a,b,controller,x,y)

    
qmod = create_model(main)

In [22]:
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/8c213254-0a37-4a2c-94b9-c43e52da760e?version=0.40.0


By executing we can actually see we get what we want:

In [17]:
def print_parsed_counts(job):
    results = job.result()
    parsed_counts = results[0].value.parsed_counts
    for parsed_state in parsed_counts: print(parsed_state.state)

job = execute(qprog)
print_parsed_counts(job)

{'controller': 0.0, 'x': 15.0, 'y': 1.0}
{'controller': 0.0, 'x': 6.0, 'y': 8.0}
{'controller': 1.0, 'x': 4.0, 'y': 9.0}
{'controller': 0.0, 'x': 8.0, 'y': 10.0}
{'controller': 0.0, 'x': 11.0, 'y': 13.0}
{'controller': 0.0, 'x': 7.0, 'y': 9.0}
{'controller': 0.0, 'x': 13.0, 'y': 15.0}
{'controller': 1.0, 'x': 0.0, 'y': 1.0}
{'controller': 1.0, 'x': 7.0, 'y': 15.0}
{'controller': 1.0, 'x': 12.0, 'y': 9.0}
{'controller': 0.0, 'x': 0.0, 'y': 2.0}
{'controller': 1.0, 'x': 10.0, 'y': 5.0}
{'controller': 1.0, 'x': 1.0, 'y': 3.0}
{'controller': 0.0, 'x': 9.0, 'y': 11.0}
{'controller': 1.0, 'x': 5.0, 'y': 11.0}
{'controller': 1.0, 'x': 6.0, 'y': 13.0}
{'controller': 1.0, 'x': 9.0, 'y': 3.0}
{'controller': 0.0, 'x': 5.0, 'y': 7.0}
{'controller': 0.0, 'x': 2.0, 'y': 4.0}
{'controller': 1.0, 'x': 11.0, 'y': 7.0}
{'controller': 0.0, 'x': 4.0, 'y': 6.0}
{'controller': 0.0, 'x': 3.0, 'y': 5.0}
{'controller': 1.0, 'x': 2.0, 'y': 5.0}
{'controller': 0.0, 'x': 1.0, 'y': 3.0}
{'controller': 0.0, 'x': 14

Of course there is the issue of rounding and overflow - when one tries to represent $2*15+1=31$ with $4$ binary digits that's not possible (because the domain $[0,31]$ of integers is represented by at least 5 bits). See our [documentation](https://docs.classiq.io/latest/user-guide/platform/qmod/python/quantum-expressions/#inplace-arithmetic-operators) for further explanations.

Let's try to use Classiq and optimize the circuit for minimal circuit width:

In [19]:
def print_depth_width(quantum_program):
    generated_circuit = QuantumProgram.parse_raw(quantum_program)
    print(f"Synthesized circuit width: {generated_circuit.data.width}, depth: {generated_circuit.transpiled_circuit.depth}")
 
qmod = set_constraints(qmod,Constraints(optimization_parameter='width'))
qprog = synthesize(qmod)
print_depth_width(qprog)

Synthesized circuit width: 20, depth: 688


And when optimizing for depth:

In [20]:
qmod = set_constraints(qmod,Constraints(optimization_parameter='depth'))
qprog = synthesize(qmod)
print_depth_width(qprog)

Synthesized circuit width: 32, depth: 398


**Firstly, we can see here a clear demonstration of the power of high-level functional design!** The same algorithm with the same functionality was optimized once for depth and once for width and the result is 2 different circuits with different characteristics that implement the same functionality.

Secondly, is this the best we can do? Obviously the Classiq synthesis engine is optimizing for us the algorithm quite good. But, can we change something with our functionality, with our algorithm to get more efficient circuits? 

If we go back to out `inplace_linear_attempt` function, we can see that we initialize a `tmp` variable that requires more qubits and is not used. For such scenarios we have the `within_apply`. This logic implements sort of $UVU^{\dagger}$ and when temporary variables are outputs of $U$ and used only by $V$ they are uncomputed by $U^{\dagger}$. Let's see for our example:

In [24]:
@qfunc
def inplace_linear_func(a:CInt,b: CInt, x:QNum, y: QNum):
    tmp = QNum('tmp')
    within_apply(compute= lambda: linear_func(a,b,x,tmp),
                action= lambda: inplace_xor(tmp,y))

With the new `control_logic`:

In [25]:
@qfunc
def control_logic_2(a: CArray[CInt], b: CArray[CInt], controller:QNum, x: QNum, y: QNum):
    
    repeat( count=a.len,         
            iteration=lambda i: control(controller==i, lambda: inplace_linear_func(a[i],b[i],x,y)))

And when we put all together now:

In [26]:
@qfunc
def main(controller: Output[QNum], x: Output[QNum],y: Output[QNum]):

    # Linear polynom parameters
    a = [1,2]
    b = [2,1]

    # Initializing x to a superposition in the domain [0,2^4-1]
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    
    #Initialize y
    allocate_num(4,False,0,y)

    # Setting the controller in a superposition
    allocate_num(1,False,0,controller)
    H(controller)

    # Implementing the control logic
    control_logic_2(a,b,controller,x,y)

    
qmod = create_model(main)

In [27]:
qprog = synthesize(qmod)

In [28]:
show(qprog)

Opening: https://platform.classiq.io/circuit/46696929-79f8-43c0-87e9-fdbebd48156f?version=0.40.0


And now when we optimize for width:

In [ ]:
qmod = set_constraints(qmod,Constraints(optimization_parameter='width'))
qprog = synthesize(qmod)
print_depth_width(qprog)

And for depth:

In [ ]:
qmod = set_constraints(qmod,Constraints(optimization_parameter='depth'))
qprog = synthesize(qmod)
print_depth_width(qprog)

So using the `within_apply` logic enabled us to reduce the optimal circuit implementation in terms of width from $20$ to $16$ and the optimal circuit depth from $398$ to $203$! 

## Cheat Sheet

### Initalizations

In [ ]:
allocate(
    num_qubits: CInt,
    out: Output[QArray[QBit, Literal["num_qubits"]]])
    '''
    x = QArray('x')
    allocate(4,x)
    '''

allocate_num(
    num_qubits: CInt,
    is_signed: QParam[bool],
    fraction_digits: CInt,
    out: Output[QNum])
'''
x = QNum('x')
allocate_num(4,False,4,x)
'''

### Operations

In [ ]:
repeat(
    count: CInt,
    iteration: QCallable[CInt],
)
'''
x = QArray
allocate(4,x)
repeat(x.len,lambda index: H(x))
'''

control(
    operand: QCallable,
    ctrl: QArray[QBit],
) 
'''
x = QArray('x')
y = QArray('y')
x = allocate(4,x)
y = allocate(4,y)
repeat(x.len,lambda i: control(lambda: X(y[i]),x[i]))
'''

# Solutions

### First Example

In [ ]:
@qfunc
def prepare_minus_state(x:QBit):
    X(x)
    H(x)
    
@qfunc
def main(x: Output[QBit]):
    allocate(1,x) # Initalize the qubit x
    prepare_minus_state(x)

quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)
show(quantum_program)

### Uniform Superposition

In [ ]:
@qfunc
def create_initial_state(reg: QArray[QBit]):
    apply_to_all(H,reg)

@qfunc
def main(reg: Output[QArray]): #TODO fill int the correct declaration here, what variables this model shoul output?
    allocate(4,reg)
    create_initial_state(reg)

qprog = synthesize(create_model(main))
show(qprog)

### Prepare Bell State

In [ ]:
@qfunc
def main(x:Output[QArray]):
    prepare_bell_state(state_num=2, q=x) # psi+

# Or:

@qfunc
def main(x: Output[QArray[QBit]]):
    prepare_state(probabilities=[0,0.5,0.5,0], bound=0.01, out=x)

model = create_model(main)
qprog = synthesize(model)
show(qprog)

### Parallel Addition

In [ ]:
@qfunc
def main(res: Output[QNum]) -> None:
    x = QNum("x")
    prepare_int(7,x)
    
    y = QNum("y")
    prepare_state(probabilities=[1/3,0,0,0,1/3,0,0,1/3], bound=0.01, out=y)
    res|=x+y

### Advanced Arithmetics

In [ ]:
@qfunc
def main(x:Output[QNum], y: Output[QNum]):

    a = 0.5
    b = 1.5
    allocate_num(num_qubits=4,is_signed=False,fraction_digits=4,out=x)
    hadamard_transform(x)
    linear_func(a,b,x,y)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)